In [1]:
from scapy.all import *
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [51]:
# OLD
def createDataFrame(pkt_lst):
    protos = []
    dst_type = []
    
    for pkt in pkt_lst:
        try:  
            ptype = {
                0x0800: 'IPv4',
                0x0806: 'ARP',
                0x8137: 'IPX',
                0x86dd: 'IPv6',
                0x88cc: 'LLDP'
            }.get(pkt.type, pkt.type)
        except AttributeError:
            ptype = 'Other'
            pass
        if ptype != 'Other':
            protos.append(ptype)
            isBcast = pkt.dst == 'ff:ff:ff:ff:ff:ff'
            if isBcast:
                dst_type.append("broadcast")
            else:
                dst_type.append("unicast")
    
    data_dict = {'protocolos': protos, 'destinos': dst_type}
    
    return pd.DataFrame.from_dict(data_dict)

def probability_chart(df):
    total =  df.shape[0]
    return (df.groupby(['simbolo']).size() / total).reset_index(name='probabilidad')

def entropy_old(df):
    probs = probability_chart(df)
    sum = 0
    for index, row in probs.iterrows():
        prob = row['probabilidad']
        sum += prob * (- math.log2(prob))
        print(prob, (- math.log2(prob)))
    return sum

def max_entropy_old(df):
    return math.log2(df.groupby(['protocolos', 'destinos']).size().reset_index().count()[0])

def plotProtocols(dframe, size=(8,8)):
    dframe.protocolos.value_counts(sort=False).plot.pie(figsize=size)

def getNoTypePkts(pkt_lst):
    pkts = []

    for pkt in pkt_lst:
        try:  
            ptype = pkt.type
        except AttributeError:
            pkts.append(pkt)
            pass
        
    return pkts
# NEW
def createDataFrame_s1(pkt_lst):
    symbol = []
    
    for pkt in pkt_lst:
        try:  
            ptype = {
                0x0800: 'IPv4',
                0x0806: 'ARP',
                0x8137: 'IPX',
                0x86dd: 'IPv6',
                0x88cc: 'LLDP'
            }.get(pkt.type, pkt.type)
        except AttributeError:
            ptype = 'Other'
            pass
        if ptype != 'Other':
            isBcast = pkt.dst == 'ff:ff:ff:ff:ff:ff'
            if isBcast:
                symbol.append(ptype + " broadcast")
            else:
                symbol.append(ptype + "  unicast")
    
    data_dict = {'simbolo': symbol}
    
    df = pd.DataFrame.from_dict(data_dict)
    
    total =  df.shape[0]
    size = df.groupby(['simbolo']).size()
    prob = size / total
    info = prob.apply(math.log2) * -1
    
    return pd.DataFrame({'simbolo':size.index, 'total':size.values, 'prob': prob.values, 'info': info.values})

def createDataFrame_s2(pkt_lst):
    arp_type = []
    srcs = []
    dsts = []
    for pkt in pkt_lst:
        if scapy.layers.l2.ARP in pkt and pkt.psrc != pkt.pdst:
            if scapy.layers.l2.ARP.is_at == pkt.op:
                arp_type.append('is-at')
            else:
                arp_type.append('who-has')
            srcs.append(pkt.psrc)
            dsts.append(pkt.pdst)

    data_dict = {'tipo': arp_type, 'src': srcs, 'dst': dsts}

    df = pd.DataFrame.from_dict(data_dict)
    df['simbolo'] = df['tipo'] + ' ' + df['dst'] 
    
    total =  df.shape[0]
    size = df.groupby(['simbolo']).size()
    prob = size / total
    info = prob.apply(math.log2) * -1
    
    return pd.DataFrame({'simbolo':size.index, 'total':size.values, 'prob': prob.values, 'info': info.values})
    
def entropy(df):
    return sum(df["prob"] * df["info"])

def max_entropy(df):
    return math.log2(sum(df.total))


In [3]:
# packets_mc = rdpcap("mc.cap")
# packets_desp = rdpcap("desp.cap")
# packets_naxio = rdpcap("pruebamac.cap")
# packets_home = rdpcap("home.cap")
# packets_mh = rdpcap("manu_home.cap")
lastDespeCap = "despe.cap"

packets = rdpcap(lastDespeCap)


In [4]:
data = createDataFrame_s1(packets)
data

,simbolo,total,prob,info
0,ARP unicast,3268,0.084447,3.565816
1,ARP broadcast,3357,0.086746,3.527052
2,IPv4 unicast,19858,0.513140,0.962576
3,IPv4 broadcast,7165,0.185147,2.433258
4,IPv6 unicast,4973,0.128505,2.960108
5,LLDP unicast,78,0.002016,8.954606


In [77]:
scapy.layers.l2.STP in getNoTypePkts(packets)[50]

True

In [52]:
entropy(data)

1.94996257796813

In [53]:
max_entropy(data)

15.24000866649586

In [45]:
data2 = createDataFrame_s2(packets)
data2

,simbolo,total,prob,info
0,is-at 10.254.32.116,6,0.000910,10.102413
1,is-at 10.254.32.148,8,0.001213,9.687376
2,is-at 10.254.32.150,114,0.017283,5.854486
3,is-at 10.254.32.154,2,0.000303,11.687376
4,is-at 10.254.32.182,1,0.000152,12.687376
5,is-at 10.254.32.191,127,0.019254,5.698691
6,is-at 10.254.32.195,7,0.001061,9.880021
7,is-at 10.254.32.221,5,0.000758,10.365448
8,is-at 10.254.32.226,9,0.001364,9.517451
9,is-at 10.254.32.26,26,0.003942,7.986936


In [54]:
entropy(data2)

5.292712920470014

In [55]:
max_entropy(data2)

12.687375683437468